#  쿼리 확장 (Query Expansion)

--- 

## 1. 개념 이해

### 1.1 쿼리 확장이란?

- 쿼리 확장(Query Expansion)은 사용자의 원본 질문을 보다 효과적인 검색을 위해 변형하거나 확장하는 기법입니다. 
- RAG 시스템에서 검색의 품질은 최종 답변의 품질을 좌우하는 핵심 요소이므로, 적절한 쿼리 확장 기법을 통해 검색 성능을 크게 향상시킬 수 있습니다.

### 1.2 쿼리 확장의 필요성

#### 🔍 **검색의 한계**
- **어휘 불일치 문제**: 사용자 질문과 문서 간 용어 차이
- **모호한 질문**: 불명확하거나 맥락이 부족한 쿼리
- **복잡한 질문**: 여러 하위 질문을 포함하는 복합 질문
- **의도 파악 어려움**: 사용자의 실제 의도와 표면적 질문의 차이

#### 🎯 **해결 방향**
- **의미론적 확장**: 동의어, 관련 개념 추가
- **구조적 분해**: 복잡한 질문을 단순한 하위 질문으로 분해
- **맥락적 확장**: 배경 지식과 일반적 개념 추가
- **가상 문서 생성**: 이상적 답변을 통한 검색 개선

### 1.3 쿼리 확장 방법론 분류

| 방법론 | 핵심 아이디어 | 장점 | 단점 | 적용 상황 |
|--------|---------------|------|------|-----------|
| **Query Reformulation** | LLM으로 질문 재작성 | 구현 간단, 즉시 적용 | 단일 변형만 생성 | 일반적인 질문 개선 |
| **Multi Query** | 다양한 관점의 질문 생성 | 검색 다양성 증가 | 계산 비용 증가 | 모호한 질문 처리 |
| **Decomposition** | 복잡한 질문을 하위 질문으로 분해 | 체계적 접근 | 분해 정확도 의존 | 복합적 질문 |
| **Step-Back Prompting** | 일반적 맥락에서 구체적 답변으로 | 포괄적 이해 | 추가 검색 필요 | 전문적/복잡한 질문 |
| **HyDE** | 가상 답변 문서 생성 | 의미적 정렬 우수 | 환각 위험 | Zero-shot 상황 |

## 환경 설정 및 준비

`(1) Env 환경변수`

In [ ]:
from dotenv import load_dotenv
load_dotenv()

`(2) 기본 라이브러리`

In [ ]:
import os
import json
from pprint import pprint
from typing import List, Dict, Any
import pandas as pd
import numpy as np

# LangChain 핵심
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser, BaseOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 검색 및 벡터 저장
from langchain_chroma import Chroma
from langchain.retrievers.multi_query import MultiQueryRetriever

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns

`(3) Langsmith tracing 설정`

In [ ]:
# Langsmith tracing 여부를 확인 (true: langsmith 추적 활성화, false: langsmith 추적 비활성화)
import os
print(os.getenv('LANGSMITH_TRACING'))

`(4) 벡터스토어 로드`

In [ ]:
def initialize_vector_store(embeddings = OpenAIEmbeddings(model="text-embedding-3-small"), collection_name="hybrid_search_db", persist_directory = "./local_chroma_db"):
    """
    기존 벡터 저장소를 로드하거나 새로 생성
    
    Returns:
        Chroma: 벡터 저장소 객체
    """
    try:
        
        # 기존 벡터 저장소 로드 시도
        vector_store = Chroma(
            collection_name=collection_name,
            embedding_function=embeddings,
            persist_directory=persist_directory,
        )
        
        doc_count = vector_store._collection.count()
        if doc_count > 0:
            print(f"✅ 기존 벡터 저장소 로드: {doc_count}개 문서")
            return vector_store
        else:
            print("⚠️ 빈 벡터 저장소입니다. 데이터를 추가해주세요.")
            return vector_store
            
    except Exception as e:
        print(f"❌ 벡터 저장소 로드 실패: {e}")
        return None

# 벡터 저장소 초기화
chroma_db = initialize_vector_store()

`(5) 백터 검색기 생성`

In [ ]:
# 기본 retriever 초기화
chroma_k_retriever = chroma_db.as_retriever(
    search_kwargs={"k": 5}
)

query = "리비안의 사업 경쟁력은 어디서 나오나요?"
retrieved_docs = chroma_k_retriever.invoke(query)

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

### 1) **Query Reformulation** 

- **Query Reformulation**은 **LLM**을 활용해 원본 질문을 다양한 형태로 재구성
- **동의어 확장**과 **키워드 추가**를 통해 검색 쿼리의 범위를 확장
- 모호한 질문을 **명확하게 구체화**하여 검색 정확도 향상
- 하나의 질문에 대해 **다양한 변형 쿼리**를 생성하여 검색 커버리지 확대

<center>
<img src="https://raw.githubusercontent.com/tsdata/image_files/main/202505/query_rewrite.png" alt="rag" align="center" border="0"  width="800" height=auto>
</center>


[출처] https://arxiv.org/abs/2305.14283

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 쿼리 리포뮬레이션을 위한 프롬프트 템플릿 정의
reformulation_template = """다음 질문을 검색 성능을 향상시키기 위해 다시 작성해주세요:
[질문]
{question}

다음 방식으로 질문을 재작성하세요:
1. 동의어 추가
2. 더 구체적인 키워드 포함
3. 관련된 개념 확장

[재작성된 질문]
"""

# 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_template(reformulation_template)

# LLM 모델 초기화
llm = ChatOpenAI(model='gpt-4.1-mini', temperature=0)

# 쿼리 리포뮬레이션 체인 생성
reformulation_chain = prompt | llm | StrOutputParser()

# 체인 실행
query = "리비안의 사업 경쟁력은 어디서 나오나요?"
reformulated_query = reformulation_chain.invoke({"question": query})

print(f"쿼리: {query}")
pprint(f"리포뮬레이션된 쿼리: \n{reformulated_query}")

In [ ]:
# 리포뮬레이션된 쿼리로 검색
retrieved_docs = chroma_k_retriever.invoke(reformulated_query)

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

In [ ]:
# Runnable 객체로 변환하여 검색기 생성 (LCEL)
reformulation_retriever = reformulation_chain | chroma_k_retriever

# 쿼리 리포뮬레이션 검색기 실행
query = "리비안의 사업 경쟁력은 어디서 나오나요?"
retrieved_docs = reformulation_retriever.invoke({"question": query})

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

In [ ]:
def load_evaluation_dataset(file_path):
    """
    평가 데이터셋 로드
    
    Args:
        file_path (str): 평가 데이터 파일 경로
    
    Returns:
        pandas.DataFrame: 평가 데이터셋
    """
    try:
        if file_path.endswith('.xlsx'):
            df = pd.read_excel(file_path)
        elif file_path.endswith('.csv'):
            df = pd.read_csv(file_path)
        else:
            raise ValueError("지원하지 않는 파일 형식")
        
        print(f"✅ 평가 데이터셋 로드: {len(df)}개 질문")
        return df
    except Exception as e:
        print(f"❌ 데이터셋 로드 실패: {e}")
        return None

# 평가 데이터셋 로드

eval_df = load_evaluation_dataset("./data/synthetic_testset.csv")
eval_df.head(3)

In [ ]:
def prepare_evaluation_data(df):
    """
    평가 데이터 전처리
    
    Args:
        df (pandas.DataFrame): 원본 데이터프레임
    
    Returns:
        tuple: (질문 리스트, 정답 문서 리스트)
    """
    questions = df['user_input'].tolist()
    
    # 정답 문서 파싱
    reference_contexts = []
    for contexts in df['reference_contexts']:
        if isinstance(contexts, str):
            # 문자열을 리스트로 변환
            context_list = eval(contexts)
        else:
            context_list = contexts
        
        # Document 객체로 변환
        docs = [Document(page_content=ctx) for ctx in context_list]
        reference_contexts.append(docs)
    
    return questions, reference_contexts

# 평가 데이터 전처리
questions, reference_contexts = prepare_evaluation_data(eval_df)

# 평가 데이터 확인
for i, (q, refs) in enumerate(zip(questions[:3], reference_contexts[:3])):
    print(f"\n[질문 {i+1}]")
    print(f"질문: {q}")
    print(f"정답 문서: {len(refs)}개")
    for j, ref in enumerate(refs):
        print(f"  [{j+1}] 내용: {ref.page_content[:50]}...")  # 내용 일부만 출력
    print("-" * 50)

# 평가 데이터 확인
for i, (q, refs) in enumerate(zip(questions[-3:], reference_contexts[-3:])):
    print(f"\n[질문 {i+1}]")
    print(f"질문: {q}")
    print(f"정답 문서: {len(refs)}개")
    for j, ref in enumerate(refs):
        print(f"  [{j+1}] 내용: {ref.page_content[:50]}...")  # 내용 일부만 출력
    print("-" * 50)

In [ ]:
# ranx-k 라이브러리 사용해서 검색 결과 평가
from ranx_k.evaluation import evaluate_with_ranx_similarity

# ranx-k 평가 실행 (embedding 점수가 높은 경우) -> 의미적 유사도 기반 평가
chroma_results = evaluate_with_ranx_similarity(
    retriever=chroma_k_retriever,
    questions=questions, 
    reference_contexts=reference_contexts,
    k=5,
    method='kiwi_rouge',  
    similarity_threshold=0.8,
)

In [ ]:
reformulation_results = evaluate_with_ranx_similarity(
    retriever=reformulation_retriever,
    questions=questions, 
    reference_contexts=reference_contexts,
    k=5,
    method='kiwi_rouge',  
    similarity_threshold=0.8,
)

### 2) **Multi Query** 

- **Multi Query**는 **Retriever의 LLM**을 사용해 단일 질문을 다수의 쿼리로 확장
- 원본 질문에 대해 **다양한 관점**과 **표현 방식**으로 쿼리 자동 생성
- **LLM의 생성 능력**을 활용해 검색 범위를 자연스럽게 확장
- 검색의 **다양성**과 **포괄성**이 향상되어 관련 문서 검색 확률 증가


<center>
<img src="https://raw.githubusercontent.com/tsdata/image_files/main/202505/multi-query.png" alt="rag" align="center" border="0"  width="800" height=auto>
</center>

[출처] https://arxiv.org/abs/2411.13154

`(1) MultiQueryRetriever 활용`

- https://python.langchain.com/docs/how_to/MultiQueryRetriever/

In [ ]:
# 멀티 쿼리 생성
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

# LLM 모델 초기화 (멀티 쿼리 생성용)
llm = ChatOpenAI(
    model='gpt-4.1-mini',
    temperature=0.7,
)

# 기본 retriever를 이용한 멀티 쿼리 생성 
multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=chroma_k_retriever, llm=llm
)

query = "리비안의 사업 경쟁력은 어디서 나오나요?"
retrieved_docs = multi_query_retriever.invoke(query)

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

`(2) Custom Prompt 활용`

In [ ]:
from typing import List

from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_core.output_parsers import BaseOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI


# 모델 초기화
llm = ChatOpenAI(model="gpt-4.1-mini")

# 출력 파서: LLM 결과를 질문 리스트로 변환
class LineListOutputParser(BaseOutputParser[List[str]]):
    """Output parser for a list of lines."""

    def parse(self, text: str) -> List[str]:
        """Split the text into lines and remove empty lines."""
        return [line.strip() for line in text.strip().split("\n") if line.strip()]
    

# 쿼리 생성 프롬프트
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""Generate three different versions of the given user question to retrieve relevant documents from a vector database. The goal is to reframe the question from various perspectives to overcome limitations of distance-based similarity search.

    The generated questions should have the following characteristics:
    1. Maintain the core intent of the original question but use different expressions or viewpoints.
    2. Include synonyms or related concepts where possible.
    3. Slightly broaden or narrow the scope of the question to potentially include diverse relevant information.

    Write each question on a new line and include only the questions.

    [Original question]
    {question}
    
    [Alternative questions]
    """,
)

# 멀티쿼리 체인 구성
multiquery_chain = QUERY_PROMPT | llm | LineListOutputParser()

# 테스트 쿼리 실행
query = "리비안의 사업 경쟁력은 어디서 나오나요?"
result = multiquery_chain.invoke({"question": query})

print("생성된 대안 질문들:")
for i, q in enumerate(result, 1):
    print(f"{i}. {q}")

In [ ]:
# 다중 쿼리 검색기 생성
multi_query_custom_retriever = MultiQueryRetriever(
    retriever=chroma_k_retriever, # 기본 retriever
    llm_chain=multiquery_chain,   # 멀티쿼리 체인
    parser_key="lines"            # "lines": 출력 파서의 키
)  

retrieved_docs = multi_query_custom_retriever.invoke(query)

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

In [ ]:
multi_query_results = evaluate_with_ranx_similarity(
    retriever=multi_query_custom_retriever,
    questions=questions, 
    reference_contexts=reference_contexts,
    k=5,
    method='kiwi_rouge',  
    similarity_threshold=0.8,
)

### 3) **Decomposition** 

- **단계별 분해 전략**을 통해 복잡한 질문을 작은 단위로 나누어 처리함
- 각 하위 질문마다 **독립적인 검색 프로세스**를 진행하여 정확도를 향상시킴
- **LEAST-TO-MOST PROMPTING**을 활용하여 체계적인 문제 해결 방식을 구현함
- 복잡한 문제를 단순화하여 검색 효율성을 극대화하는 방법론

<center>
<img src="https://raw.githubusercontent.com/tsdata/image_files/main/202505/query_decomposition.png" alt="rag" align="center" border="0"  width="800" height=auto>
</center>


[출처] https://arxiv.org/pdf/2205.10625

In [ ]:
from langchain.prompts import PromptTemplate
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to decompose the given input question into multiple sub-questions. 
    The goal is to break down the input into a set of sub-problems/sub-questions that can be answered independently.

    Follow these guidelines to generate the sub-questions:
    1. Cover various aspects related to the core topic of the original question.
    2. Each sub-question should be specific, clear, and answerable independently.
    3. Ensure that the sub-questions collectively address all important aspects of the original question.
    4. Consider temporal aspects (past, present, future) where applicable.
    5. Formulate the questions in a direct and concise manner.

    [Input question] 
    {question}

    [Sub-questions (5)]
    """,
)

# 쿼리 생성 체인
decomposition_chain = QUERY_PROMPT | llm | LineListOutputParser()

# 테스트 쿼리 실행
query = "리비안의 사업 경쟁력은 어디서 나오나요?"
result = decomposition_chain.invoke({"question": query})

print("생성된 서브 질문들:")
for i, q in enumerate(result, 1):
    print(f"{i}. {q}")

In [ ]:
# 다중 쿼리 검색기 생성
multi_query_decompostion_retriever = MultiQueryRetriever(
    retriever=chroma_k_retriever,    # 기본 retriever
    llm_chain=decomposition_chain,   # 서브 질문 생성 체인
    parser_key="lines"               # "lines": 출력 파서의 키
)  

retrieved_docs = multi_query_decompostion_retriever.invoke(query)

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

In [ ]:
decomposition_results = evaluate_with_ranx_similarity(
    retriever=multi_query_decompostion_retriever,
    questions=questions, 
    reference_contexts=reference_contexts,
    k=5,
    method='kiwi_rouge',  
    similarity_threshold=0.8,
)

### 4) **Step-Back Prompting**

- **단계적 후퇴 방식**을 통해 구체적 질문을 일반적 맥락에서 접근함
- **맥락 기반 검색**으로 넓은 관점에서 구체적 답변으로 좁혀나감
- **포괄적 접근법**을 활용하여 복잡한 질문에 대한 이해도를 높임
- 일반적 맥락에서 시작하여 구체적 해답을 찾아가는 체계적 접근 방식

<center>
<img src="https://raw.githubusercontent.com/tsdata/image_files/main/202505/query_stepback.png" alt="rag" align="center" border="0"  width="800" height=auto>
</center>


[출처] https://arxiv.org/pdf/2310.06117

`(1) Step-Back 질문 생성`

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# Few Shot 예제 - (구체적 질문, 포괄적 질문) 쌍
examples = [
    {
        "input": "애플의 M1 칩 개발이 기업 가치에 미친 영향은?",
        "output": "기업의 핵심 기술 내재화가 경쟁우위에 미치는 영향은 무엇인가?",
    },
    {
        "input": "아마존의 AWS가 수익성에 기여하는 방식은?",
        "output": "기업의 새로운 사업 영역 확장이 수익 구조에 미치는 영향은 무엇인가?",
    },
    {
        "input": "토요타의 하이브리드 기술 전략의 핵심은?",
        "output": "자동차 산업에서 친환경 기술 혁신이 기업 성장에 미치는 영향은 무엇인가?",
    }
]

# 프롬프트 템플릿 초기화
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}"),
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# Step-Back 생성을 위한 프롬프트
step_back_prompt = ChatPromptTemplate.from_messages([
            (
                "system",
                """당신은 기업 분석 전문가입니다. 특정 기업에 대한 구체적인 질문을 해당 산업이나 비즈니스 전반의 일반적인 관점에서 
                재해석하는 것이 임무입니다. 산업 동향, 경쟁 구도, 기술 혁신, 사업 모델 등의 관점에서 더 포괄적인 질문으로 
                바꾸어 주세요. 다음은 예시입니다:"""
            ),
            few_shot_prompt,
            ("user", "{question}"),
        ])

# Step-Back 체인 생성
step_back_chain = step_back_prompt | llm | StrOutputParser()

# Step-Back 질문 생성
query = "리비안의 사업 경쟁력은 어디서 나오나요?"
step_back_question = step_back_chain.invoke({"question": query})

print(f"쿼리: {query}")
print(f"Step-Back 질문: {step_back_question}")

In [ ]:
# Step-Back 검색기 생성
step_back_retriever = step_back_chain | chroma_k_retriever

# Step-Back 검색 실행
retrieved_docs = step_back_retriever.invoke({"question": query})

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

In [ ]:
stepback_results = evaluate_with_ranx_similarity(
    retriever=step_back_retriever,
    questions=questions, 
    reference_contexts=reference_contexts,
    k=5,
    method='kiwi_rouge',  
    similarity_threshold=0.8,
)

`(2) 최종 답변 생성`

In [ ]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate


# 프롬프트 템플릿 초기화
response_prompt = ChatPromptTemplate.from_template(
            """당신은 전문가입니다. 다음 컨텍스트와 질문을 바탕으로 포괄적인 답변을 제공해주세요.

            일반 컨텍스트:
            {normal_context}
            
            기본 개념 컨텍스트:
            {step_back_context}
            
            원래 질문: {question}
            
            답변:"""
        )

# 문서 포맷팅 함수
def format_docs(docs):
    return "\n".join([doc.page_content for doc in docs])


# 답변 생성 체인
answer_chain = (
            {
                "normal_context": chroma_k_retriever,
                "step_back_context": step_back_retriever,
                "question": RunnablePassthrough(),
            }
            | response_prompt
            | llm
            | StrOutputParser()
        )

# 답변 생성
query = "리비안의 사업 경쟁력은 어디서 나오나요?"
answer = answer_chain.invoke(query)

print(f"쿼리: {query}")
print(f"답변: {answer}")

### 5) **HyDE** (Hypothetical Document Embedding)

- **가상 문서 생성**을 통해 주어진 질문에 대해 가상의 이상적인 답변 문서를 LLM으로 생성함
- 생성된 문서의 **임베딩 기반 검색**으로 실제 문서와 매칭을 수행함
- **맥락 기반 검색 방식**으로 질문의 의도를 더 정확하게 반영함

<center>
<img src="https://raw.githubusercontent.com/tsdata/image_files/main/202505/query_HyDE.png" alt="rag" align="center" border="0"  width="800" height=auto>
</center>


[출처] https://arxiv.org/abs/2212.10496

`(1) 가상 문서 생성`

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# HyDE를 위한 프롬프트 템플릿 생성
template = """주어진 질문에 대한 이상적인 문서 내용을 생성해주세요.
문서는 학술적이고 전문적인 톤으로 작성되어야 합니다.

질문: {question}

문서 내용:"""

hyde_prompt = ChatPromptTemplate.from_template(template)

# LLM 모델 초기화
hyde_llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# 문서 생성 체인 생성
hyde_chain = hyde_prompt | hyde_llm | StrOutputParser()

# 문서 생성 실행
query = "리비안의 사업 경쟁력은 어디서 나오나요?"
hypothetical_doc = hyde_chain.invoke({"question": query})

print(f"쿼리: {query}")
print(f"문서 내용: {hypothetical_doc}")

`(2) 유사 문서 검색`

In [ ]:
# 가상 문서를 기반으로 실제 문서 검색
    
retrieved_docs = chroma_k_retriever.invoke(hypothetical_doc)

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

`(3) HyDE 검색기 생성`

In [ ]:
hyde_retriever = hyde_chain | chroma_k_retriever

retrieved_docs = hyde_retriever.invoke({"question": query})

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

`(3) 최종 답변 생성`

In [ ]:
# 최종 RAG를 위한 프롬프트 템플릿 생성
template = """다음 컨텍스트를 바탕으로 질문에 답변해주세요:

컨텍스트:
{context}

질문: {question}

답변:"""

rag_prompt =  ChatPromptTemplate.from_template(template)

# RAG 체인 생성
rag_llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
rag_chain = rag_prompt | rag_llm | StrOutputParser()
    
# RAG 실행
query = "리비안의 사업 경쟁력은 어디서 나오나요?"
context = format_docs(retrieved_docs)

answer = rag_chain.invoke({"context": context, "question": query})

print(f"쿼리: {query}")
print(f"답변: {answer}")

`(4) HyDE 체인 종합`

In [ ]:
# Step 1. 가상 문서 생성 및 검색
query = "테슬라의 경영진을 분석해주세요."

# Step 2. 유사 문서 검색
retrieved_docs = hyde_retriever.invoke({"question": query})

# Step 3. 최종 답변 생성
final_answer = rag_chain.invoke(
    {
        "context": format_docs(retrieved_docs), 
        "question": query
    }
)

print(f"쿼리: {query}")
print(f"가상 문서 내용: {hypothetical_doc}")
print(f"답변: {final_answer}")